# インストール

In [1]:
%%capture
!pip install bertopic

# データセットの準備：[参考サイト](https://qiita.com/sugulu_Ogawa_ISID/items/697bd03499c1de9cf082)

In [2]:
# NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968に対する一時的な対策
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

--2023-06-26 12:01:39--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  2.68MB/s    in 3.1s    

2023-06-26 12:01:43 (2.68 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [4]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)


['it-life-hack', 'kaden-channel', 'README.txt', 'livedoor-homme', 'CHANGES.txt', 'peachy', 'smax', 'topic-news', 'dokujo-tsushin', 'movie-enter', 'sports-watch']
カテゴリー数: 9
['it-life-hack', 'kaden-channel', 'livedoor-homme', 'peachy', 'smax', 'topic-news', 'dokujo-tsushin', 'movie-enter', 'sports-watch']


In [5]:
# 本文を取得する前処理関数を定義
def extract_main_txt(file_name):
    with open(file_name, encoding='utf-8') as text_file: # utf-8を追加
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text

In [6]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))
    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)


In [7]:
# 0番目の文章とラベルを確認
print(list_text[0])
print(list_label[0])

本日8月16日、22時をもって情報開示が解禁されたのがNVIDIAの新型GPU「GeForce GTX 660 Ti」だ。1344ユニットのCUDAコア、192ビットのメモリインターフェイスを搭載、アーキテクチャは当然Keplerである。自動オーバークロック機能の「GPU Boost」により、GPUのポテンシャルを常に発揮し続ける設計で、パフォーマンス、電力効率ともに大変優れる製品になっている。ドスパラなどは、この発表に合わせて深夜販売を行った。今後のGPUのメインストリームとなっていくと思われ市場での価格はノーマル版で3万円台の前半、クロックアップ版が3万円台の半ばくらいに設定される。この発表に合わせてZOTACやMSI、Gainwardといったメーカーから、GeForce GTX 660 Ti搭載製品が投入された。各メーカーの製品名と価格を紹介しよう。■ZOTAC（アスク取り扱い）ZOTAC GeForce GTX 660 Ti 2GB製品名：ZOTAC GeForce GTX 660 Ti 2GB型番：ZTGTX660Ti-2GD5R0/ZT-60802-10P発売時期：8月16日予想市場価格：33,000円前後ZOTAC GeForce GTX 660 Ti AMP Edition製品名：ZOTAC GeForce GTX 660 Ti AMP Edition型番：ZTGTX660Ti-2GD5AMPR0/ZT-60804-10P発売時期 8月16日予想市場価格： 38,000円前後■MSI（アスク取り扱い）MSI N660GTX-Ti Twin Frozr IV PE OC製品名：MSI N660GTX-Ti Twin Frozr IV PE OC型番：N660GTX-Ti Twin Frozr IV PE OC発売時期：8月16日予想市場価格：33,000円前後■Gainward（エムヴィケー扱い）GW GTX660TI 2GBD5 PHANTOM製品名：GW GTX660TI 2GBD5 PHANTOM発売時期：今週中予想市場価格：32,980円前後GW GTX660TI 2GBD5製品名：GW GTX660TI 2GBD5発売時期：今週中予想市場価格：30,980円前後■アスク■エムヴィケー
it-life-hack


# BERTopicの予測

In [8]:
from bertopic import BERTopic
from sklearn.cluster import KMeans
cluster_model = KMeans(n_clusters=9)

topic_model = BERTopic(language="japanese", hdbscan_model=cluster_model, calculate_probabilities=True, verbose=True)
topics, probs = topic_model.fit_transform(list_text)

Batches:   0%|          | 0/231 [00:00<?, ?it/s]

2023-06-26 12:03:00,813 - BERTopic - Transformed documents to Embeddings
2023-06-26 12:03:38,443 - BERTopic - Reduced dimensionality
2023-06-26 12:03:38,622 - BERTopic - Clustered reduced embeddings


## 頻度ごとにトピックを見る

In [9]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,0,1144,0_また_00_しかし_peachy,"[また, 00, しかし, peachy, 話題, では, そして, でも, 関連リンク, ...",[今月もこんにちは。独女の皆様。お下劣、毒舌、さあおやつ！ドロンジョーヌ恩田です。熱中症にビ...
1,1,1123,1_映画_movie_公式サイト_そして,"[映画, movie, 公式サイト, そして, また, アベンジャーズ, 特集, 関連記事,...",[『エイリアン』『グラディエーター』など、映画史上において幾多の金字塔を打ち立ててきた“生き...
2,2,949,2_など_関連情報_関連記事_また,"[など, 関連情報, 関連記事, また, ネット掲示板では, livedoor, 小沢, 話...",[連日、国家に対するデモ・暴動が相次いでいるギリシャでは、16日、暴徒化したギリシャ人が街中...
3,3,909,3_でも_type_しかし_livedoor,"[でも, type, しかし, livedoor, オフィスエムツー, また, 仮名, 情報...",[「3年で転職は早すぎる？」「将来が見えない」「仕事が面白くない」・・・若手社会人の悩みは尽...
4,4,886,4_では_また_だが_s1,"[では, また, だが, s1, tbs, news, 日本テレビ, と語り, 週刊アサヒ芸...",[11日深夜放送、TBS「S1」では「反骨の43歳 金本知憲 復活へ知られざる激闘録」と題し...


In [10]:
topic_model.get_topic(0)  # Select the most frequent topic

[('また', 0.013741838170814485),
 ('00', 0.009903689804160004),
 ('しかし', 0.007720596234312432),
 ('peachy', 0.007654122040688737),
 ('話題', 0.0071133875935020545),
 ('では', 0.006904503792356612),
 ('そして', 0.006840518169957998),
 ('でも', 0.0064143591725005535),
 ('関連リンク', 0.006389133232855914),
 ('さらに', 0.005750516976569415)]

## ドキュメントごとにトピックなど様々な情報を出力

In [11]:
topic_model.get_document_info(list_text)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document
0,本日8月16日、22時をもって情報開示が解禁されたのがNVIDIAの新型GPU「GeForc...,7,7_話題_ビデオsalon_デジ通_関連記事,"[話題, ビデオsalon, デジ通, 関連記事, 売れ筋チェック, また, gv, int...",[今回のテーマはピッチとスピードです。アマチュアのナレーションをすくうテクニックが隠されてい...,話題 - ビデオsalon - デジ通 - 関連記事 - 売れ筋チェック - また - gv...,False
1,PS Vitaを購入する際、Wi-Fiモデルか3G/Wi-Fiモデルのどちらかを選択する必要...,7,7_話題_ビデオsalon_デジ通_関連記事,"[話題, ビデオsalon, デジ通, 関連記事, 売れ筋チェック, また, gv, int...",[今回のテーマはピッチとスピードです。アマチュアのナレーションをすくうテクニックが隠されてい...,話題 - ビデオsalon - デジ通 - 関連記事 - 売れ筋チェック - また - gv...,False
2,グリーは2012年2月20日、同社が運営する一部のソーシャルゲームにおいて、通常の想定数を超...,5,5_max_iphone_話題_store,"[max, iphone, 話題, store, エスマックス, play, com, デジ...",[おでんはやっぱり大根！？突然、おでんが食べたくなる季節……ではないですが、染み込んだ大根や...,max - iphone - 話題 - store - エスマックス - play - co...,False
3,アスクは、同社が取り扱うSapphire Technology社製グラフィックスボードの購入...,5,5_max_iphone_話題_store,"[max, iphone, 話題, store, エスマックス, play, com, デジ...",[おでんはやっぱり大根！？突然、おでんが食べたくなる季節……ではないですが、染み込んだ大根や...,max - iphone - 話題 - store - エスマックス - play - co...,False
4,Twitterの伝搬力の強さを思い知るユニークな出来事が話題になっている。5月8日に東京大学...,8,8_話題_虎の巻_知っ得_をクリックする,"[話題, 虎の巻, 知っ得, をクリックする, 関連記事, kobo, 売れ筋チェック, ま...",[Facebookのタイムラインページはみんな同じ感じでちょっとつまらないなぁなんて思ってい...,話題 - 虎の巻 - 知っ得 - をクリックする - 関連記事 - kobo - 売れ筋チェ...,False
...,...,...,...,...,...,...,...
7371,11日深夜、日本テレビ「Going! Sports＆News」では、先の競泳パンパシフィック...,4,4_では_また_だが_s1,"[では, また, だが, s1, tbs, news, 日本テレビ, と語り, 週刊アサヒ芸...",[11日深夜放送、TBS「S1」では「反骨の43歳 金本知憲 復活へ知られざる激闘録」と題し...,では - また - だが - s1 - tbs - news - 日本テレビ - と語り -...,False
7372,7月10日、豪州のキャンベラ・ナショナルコンベンションセンターで開催されたK-1オセアニアG...,4,4_では_また_だが_s1,"[では, また, だが, s1, tbs, news, 日本テレビ, と語り, 週刊アサヒ芸...",[11日深夜放送、TBS「S1」では「反骨の43歳 金本知憲 復活へ知られざる激闘録」と題し...,では - また - だが - s1 - tbs - news - 日本テレビ - と語り -...,False
7373,韓国の全州で行われた、アジア・チャンピオンズリーグ準々決勝第2戦では、セレッソ大阪が敵地で全...,4,4_では_また_だが_s1,"[では, また, だが, s1, tbs, news, 日本テレビ, と語り, 週刊アサヒ芸...",[11日深夜放送、TBS「S1」では「反骨の43歳 金本知憲 復活へ知られざる激闘録」と題し...,では - また - だが - s1 - tbs - news - 日本テレビ - と語り -...,False
7374,28日現在、リーグ最下位に低迷する東北楽天ゴールデンイーグルス。田中将大、岩隈久志の二大エー...,4,4_では_また_だが_s1,"[では, また, だが, s1, tbs, news, 日本テレビ, と語り, 週刊アサヒ芸...",[11日深夜放送、TBS「S1」では「反骨の43歳 金本知憲 復活へ知られざる激闘録」と題し...,では - また - だが - s1 - tbs - news - 日本テレビ - と語り -...,False


## 2次元で可視化

In [12]:
topic_model.visualize_topics(top_n_topics = 9)

## 階層的クラスタリング

In [18]:
topic_model.visualize_hierarchy(top_n_topics=9)

## トピックツリーの可視化

In [19]:
hierarchical_topics = topic_model.hierarchical_topics(list_text)
topic_model.get_topic_tree(hierarchical_topics)

100%|██████████| 8/8 [00:00<00:00, 35.66it/s]


'.\n├─また_など_映画_しかし_関連記事\n│    ├─映画_また_しかし_でも_そして\n│    │    ├─■──映画_movie_公式サイト_そして_また ── Topic: 1\n│    │    └─また_でも_しかし_livedoor_そして\n│    │         ├─■──また_00_しかし_peachy_話題 ── Topic: 0\n│    │         └─■──でも_type_しかし_livedoor_オフィスエムツー ── Topic: 3\n│    └─など_関連情報_また_関連記事_では\n│         ├─■──など_関連情報_関連記事_また_ネット掲示板では ── Topic: 2\n│         └─■──では_また_だが_s1_tbs ── Topic: 4\n└─max_エスマックス_話題_phone_on\n     ├─話題_関連記事_ビデオsalon_デジ通_売れ筋チェック\n     │    ├─■──話題_虎の巻_知っ得_をクリックする_関連記事 ── Topic: 8\n     │    └─■──話題_ビデオsalon_デジ通_関連記事_売れ筋チェック ── Topic: 7\n     └─max_エスマックス_phone_on_twitter\n          ├─■──max_iphone_話題_store_エスマックス ── Topic: 5\n          └─■──max_エスマックス_phone_on_smaxjp ── Topic: 6\n'

## トピックごとのワードスコア

In [20]:
topic_model.visualize_barchart(top_n_topics=9)

## ヒートマップ

In [24]:
topic_model.visualize_heatmap(width=1000, height=1000)

## Termのスコアの低下

In [25]:
topic_model.visualize_term_rank()

## トピックの検索

In [28]:
similar_topics, similarity = topic_model.find_topics("アニメ", top_n=5); similar_topics

[1, 2, 0, 7, 8]

In [29]:
topic_model.get_topic(similar_topics[0])

[('映画', 0.031062352645799807),
 ('movie', 0.018657081474495397),
 ('公式サイト', 0.012257715763722133),
 ('そして', 0.010565870303826348),
 ('また', 0.010463272416064453),
 ('アベンジャーズ', 0.009865150469871123),
 ('特集', 0.009466112094939838),
 ('関連記事', 0.008802270114460246),
 ('enter', 0.008785620251602096),
 ('本作は', 0.008691671791300799)]